In [1]:
using Pkg
Pkg.activate("C:\\Users\alili\\.julia\\environments\\v1.5\\Project.toml") # change path 
Pkg.instantiate()

 Activating new environment at `C:\Userslili\.julia\environments\v1.5\Project.toml`
   Updating registry at `C:\Users\alili\.julia\registries\General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %                                   ]  7.5 %====>                                    ]  9.0 %======>                                  ]  13.3 %Fetching: [=========>                               ]  21.7 %]  24.7 % [===========>                             ]  25.6 % [============>                            ]  27.9 %Fetching: [============>                            ]  29.7 %Fetching: [=============>                           ]  30.7 % %]  42.8 %Fetching: [===================>                     ]  46.5 % [====================>                    ]  49.5 %======================>                  ]  54.2 % % [========================>                ]  59.6 %>               ]  61.7 %==========================>              ]  64.3 %>             ]  66.4 %>            ]  68.2 %         ]  77.2 %>   ]  91.6 % [========================================>]  97.8 %]  99.1 %

LoadError: IOError: mkdir: invalid argument (EINVAL)

In [2]:
using JuMP, LinearAlgebra, Plots, StatsPlots, CPLEX, Statistics, CSV, DataFrames

In [5]:
#read scenario data
scenarios_hexagons = CSV.read("scenarios_multinomial.csv", DataFrame)
scenarios_prices = CSV.read("priceScenarions3.csv", DataFrame)

SOC = 0.33
hexagons = names(scenarios_hexagons)[1:length(names(scenarios_hexagons))-1] # vector of hexagons (strings)
hours = names(scenarios_prices)[3:length(names(scenarios_prices))] #vector of hours for charging (strings)
car_kWh = 30 # capacity of each car

#demand mxn (m=hexagon, n=scenario) in kWh summe über car*max_kWh*(1-SOC)
demand_cars = transpose(convert(Matrix,scenarios_hexagons[:,hexagons]))
demand_kWh = demand_cars.*car_kWh.*(1-SOC)

#prices mxn (m=hours, n=scenario)
prices = transpose(convert(Matrix,scenarios_prices[:,hours]))

probability_hexagons = transpose(scenarios_hexagons.probs) # probabilities for each scenario (#cars per hexagon)
probability_prices = transpose(scenarios_prices.Probability) # probabilities for each scenario (timeseries prices)

P = [3, 7, 11] # different charger types in kW
charger_cost = [1500, 800, 500] # investment cost for charger 3 kW, 7 kW und 11 kW in €/kW
opportunity_cost = 5000 # cost of demand not served


5000

In [11]:
function charger_siting(SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, prices, 
        probability_hexagons, probability_prices, P, charger_cost, opportunity_cost)
    
    # Define Sets
    Γ = Array{Int}(1:length(hexagons)) # Set with hexagons
    Σ = Array{Int}(1:length(P)) # Set with charger types 
    Ω = Array{Int}(1:length(probability_hexagons)) # Set with scenarios for hexagons
    Τ = Array{Int}(1:length(hours)) # Set with hours
    Ρ = Array{Int}(1:length(probability_prices)) # Set with scenarios for prices

    println(Σ)
    
    # Initialize model    
    m = Model(CPLEX.Optimizer)

    # Define variables
    @variable(m, 0 <= x_charger[i=Γ,j=Σ],Int) # number of specific charger type in each hexagon
    @variable(m, 0 <= E_supplied[i=Γ,j=Σ,k=Ω,t=Τ,p=Ρ]) # supplied energy in each hexagon kWh
    @variable(m, 0 <= E_not_supplied[i=Γ,k=Ω,p=Ρ])     # demand not covered kWh
    @variable(m, 0 <= cars_supplied[i=Γ,j=Σ,k=Ω,p=Ρ],Int)
    @variable(m, 0 <= cars_not_supplied[i=Γ,j=Σ,k=Ω,p=Ρ],Int)

    # Objective function 
    @expression(m, invest_cost[i=Γ,j=Σ], charger_cost[j].*x_charger[i,j].*P[j]) # first stage investment cost
    @expression(m, charging_opportunity_cost[i=Γ,k=Ω,p=Ρ], opportunity_cost.*E_not_supplied[i,k,p]
                                                     .*probability_hexagons[k]
                                                     .*probability_prices[p])
    @expression(m, grid_usage[i=Γ,k=Ω,t=Τ,p=Ρ], 0.1*sum(E_supplied[i,j,k,t,p] for j in Σ)^2
                                                     .*probability_hexagons[k]
                                                     .*probability_prices[p])
    @expression(m, charging_cost[i=Γ,j=Σ,k=Ω,t=Τ,p=Ρ], prices[t,p].*E_supplied[i,j,k,t,p]
                                                     .*probability_hexagons[k]
                                                     .*probability_prices[p]) #minimize cost of charging

    @objective(m, Min, sum(invest_cost) 
                     + sum(charging_opportunity_cost)
                     + sum(grid_usage)
                     + sum(charging_cost)
                      )

    # Constraints
    # Eq. Balance equation: Demand in each hexagon has to equal demand. Loss of load is possible
    @constraint(m, eq_balance[i=Γ,k=Ω,p=Ρ], sum(sum(E_supplied[i,j,k,t,p] for j in Σ) for t in Τ) + E_not_supplied[i,k,p] == demand_kWh[i,k])

    # Eq. energy limit: rated_power*chargingtime must not exeed energy
    @constraint(m, eq_cap[i=Γ,j=Σ,k=Ω,t=Τ,p=Ρ], E_supplied[i,j,k,t,p] <= x_charger[i,j].*P[j])

    # Eq. supplied cars and not supplied cars have to equal amount of cars in each hexagon
    @constraint(m, eq_cars_supplied[i=Γ,j=Σ,k=Ω,p=Ρ], sum(cars_supplied[i,j,k,p] for j in Σ) 
                                                    + sum(cars_not_supplied[i,j,k,p] for j in Σ)
                                                   == demand_cars[i,k])

    # Eq. maximum one car per charging station
    @constraint(m, eq_max_cars[i=Γ,j=Σ,k=Ω,p=Ρ], cars_supplied[i,j,k,p] <= x_charger[i,j])

    # Eq. 
    @constraint(m, eq_max_supply[i=Γ,j=Σ,k=Ω,p=Ρ], sum(E_supplied[i,j,k,t,p] for t in Τ) <= cars_supplied[i,j,k,p]
                                                                                         .* car_kWh.*(1-SOC))


    # Run optimization
    optimize!(m)

    # Evaluate results
    obj = objective_value(m)
    charger_in_hex = value.(x_charger)
    
    return charger_in_hex, x_charger, E_not_supplied, E_supplied, cars_not_supplied, cars_supplied, obj
end


charger_siting (generic function with 1 method)

In [12]:
charger_in_hex, x_charger, E_not_supplied, E_supplied, cars_not_supplied, cars_supplied, obj= charger_siting(
    SOC, hexagons, hours, car_kWh, demand_cars, demand_kWh, prices, 
        probability_hexagons, probability_prices, P, charger_cost, opportunity_cost)

println(charger_in_hex)
println(obj)

[1, 2, 3]
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
Found incumbent of value 5.9888580e+07 after 0.02 sec. (3.96 ticks)
Tried aggregator 1 time.
MIQP Presolve eliminated 17256 rows and 15861 columns.
Reduced MIQP has 30856 rows, 27886 columns, and 104272 nonzeros.
Reduced MIQP has 704 binaries, 3774 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 67032 nonzeros.
Presolve time = 0.25 sec. (68.56 ticks)
Probing time = 0.05 sec. (2.04 ticks)
Tried aggregator 1 time.
MIQP Presolve eliminated 116 rows and 107 columns.
Reduced MIQP has 30740 rows, 27779 columns, and 103880 nonzeros.
Reduced MIQP has 688 binaries, 3771 generals, 0 SOSs, and 0 indicators.
Reduced MIQP objective Q matrix has 66780 nonzeros.
Presolve time = 0.11 sec. (45.47 ticks)
Classifier predicts products in MIQP should be linearized.
Probing time = 0.05 sec. (2.01 ticks)
Clique table members: 172.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Paral